# Python para finanças - risco de ações

## Importação das bibliotecas e base de dados

In [ ]:
import pandas as pd
import numpy as np
import math
from scipy import stats
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from get_stocks_yfinance import get_stocks_yfinance
from IPython.display import HTML
from constants import stocks

In [ ]:
dataset, days = get_stocks_yfinance(stocks)
start_date = dataset.index[0]
end_date = dataset.index[-1]

In [ ]:
dataset

In [ ]:
dataset.describe()

## Cálculo dos retornos logarítmicos anuais
### Utilizado para taxas da mesma ação em períodos diferentes, caso seja comparativo entre ações utilizar taxa de retorno simples

In [ ]:
# ? new data frame
groupped_df = pd.DataFrame()
# ? one line for each year
groupped_df.index = dataset.index.year.unique()
# ? for each column (company) one rate list
for column in dataset.columns:
    rates = []
    # ? for each year, get the first and the last day of the year and takes log return rate
    for y in dataset.index.year.unique():
        first_of_year = dataset.index[(dataset.index.year == y)].min()
        last_of_year = dataset.index[(dataset.index.year == y)].max()
        rates.append(np.log(dataset.loc[last_of_year, column] / dataset.loc[first_of_year, column]))
    # ? insert the column
    groupped_df[column] = rates

### Formatando em %

In [ ]:
groupped_df.applymap(lambda value: f"{value:.2%}")

### Gráfico de comparação

In [ ]:
fig = px.line()
for column in groupped_df.columns:
    fig.add_scatter(x=groupped_df.index, y=groupped_df[column], name=column).update_layout(hovermode="x")
fig.show()

## Variância
### Quadrado do desvio padrão

In [ ]:
(groupped_df * 100).var().sort_values(ascending=False)

### Em um período específico

In [ ]:
dataset[dataset.index.year == 2023][dataset.columns[1]].var()

## Desvio padrão
### O quanto os dados variam em relação a média

In [ ]:
(groupped_df * 100).std().sort_values(ascending=False)

## Coeficiente de variação
### Indica variação em %

In [ ]:
(groupped_df.apply(stats.variation) * 100).sort_values(ascending=False)

## Cálculos com todas as ações - risco médio anual

### Taxa de retorno simples diária

In [ ]:
daily_rr = (dataset / dataset.shift(1)) - 1
daily_rr

### Desvio padrão para todo o período em %

In [ ]:
(daily_rr.std() * 100).sort_values(ascending=False)

### Média de dias em que as bolsas de valores para essas ações ficaram abertas ao longo dos anos da base de dados

In [ ]:
DAYS = int(len(dataset) / len(dataset.index.year.unique()))
DAYS

### Desvio padrão médio anual

In [ ]:
(daily_rr.std() * DAYS).sort_values(ascending=False)

### Desvio padrão é a raiz quadrada da variância, para se ter um valor anual mais "justo", é necessário retirar a raiz quadrada do número de dias no ano

In [ ]:
(daily_rr.std() * math.sqrt(DAYS)).sort_values(ascending=False)

## Correlação entre ações

### Covariância

In [ ]:
daily_rr.cov()

### Correlação

In [ ]:
daily_rr.corr()

### Heatmap

#### Plotly

In [ ]:
px.imshow(daily_rr.corr().applymap(lambda x: float(f"{x:.2f}")), text_auto=True, width=1000, height=700)

#### Seaborn

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(daily_rr.corr(), annot=True);

## Risco de um portfólio

### Dataset taxas de retorno diárias

In [ ]:
daily_rr

### Todas as ações

In [ ]:
dataset.columns

### Pesos

In [ ]:
weights = np.array([
    0.0,
    0.70,
    0.0,
    0.30,
])

w_sum = weights.sum() # soma para garantir que está em 1.0
annual_cov = daily_rr.cov() * DAYS # covariancia anual
portfolio_var = np.dot(weights, np.dot(daily_rr.cov() * DAYS, weights)) # Variância da carteira contendo todas as empresas
portfolio_volatility = math.sqrt(portfolio_var) # Volatilidade de todo o portfolio %

print(f"Weights sum: {w_sum}")             
print(f"Anual covariance: {annual_cov}") 
print(f"Portolio variation: {portfolio_var}") 
print(f"Portfolio volatility: {portfolio_volatility}")

## Risco sistemático e não sistemático

- Risco sistemático: mudança diária no preço das ações, devido a eventos como recessão da economia, comportamento do cliente (eventos externos). Não pode ser eliminado e pode afetar todas as empresas
- Risco não sistemático: Eventos específicos na empresa (depende do tipo de empresa)

### Variância anual

In [ ]:
daily_rr.var() * DAYS

### Variância anual com base nos pesos, ou seja, alocação de portfolio

In [ ]:
weights_variation = (daily_rr.var() * DAYS) * weights
weights_variation

### Subtração de todos os valores resultantes de variância anual X pesos

In [ ]:
sub = np.sum([-weight for weight in weights_variation])
sub

### Risco não sistemático

In [ ]:
unsystematic_risk1 = (portfolio_var - sub)
unsystematic_risk1